In [ ]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

batch_size = 128
epochs = 10
seed = 1
log_interval = 10
torch.manual_seed(seed)
device = "cpu"

In [ ]:
from Tars.distributions import NormalModel, BernoulliModel
from Tars.models import VI
from Tars.utils import get_dict_values

In [ ]:
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
x_dim = 784
a_dim = 64
z_dim = 64

# inference models
class Q1(NormalModel):
    def __init__(self):
        super(Q1, self).__init__(cond_var=["x"], var=["a"])

        self.fc1 = nn.Linear(x_dim, 400)
        self.fc21 = nn.Linear(400, a_dim)
        self.fc22 = nn.Linear(400, a_dim)

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), F.softplus(self.fc22(h1))
    
class Q2(NormalModel):
    def __init__(self):
        super(Q2, self).__init__(cond_var=["x"], var=["z"])

        self.fc1 = nn.Linear(x_dim, 400)
        self.fc21 = nn.Linear(400, z_dim)
        self.fc22 = nn.Linear(400, z_dim)

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), F.softplus(self.fc22(h1))    
    

# generative models
loc = torch.tensor(0.).to(device)
scale = torch.tensor(1.).to(device)
p1 = NormalModel(loc=loc, scale=scale, var=["z"], dim=z_dim)
    
class P2(NormalModel):
    def __init__(self):
        super(P2, self).__init__(cond_var=["z"], var=["a"])

        self.fc1 = nn.Linear(z_dim, 400)
        self.fc21 = nn.Linear(400, a_dim)
        self.fc22 = nn.Linear(400, a_dim)

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), F.softplus(self.fc22(h1))
    
class P3(BernoulliModel):
    def __init__(self):
        super(P3, self).__init__(cond_var=["a"], var=["x"])

        self.fc3 = nn.Linear(a_dim, 400)
        self.fc4 = nn.Linear(400, x_dim)

    def forward(self, x):
        h3 = F.relu(self.fc3(x))
        return F.sigmoid(self.fc4(h3))    
    
    
q1 = Q1()
q2 = Q2() 

p2 = P2()
p3 = P3()

In [ ]:
q = q1 * q2
_p = p2 * p3
p = _p * p1
p.to(device)
q.to(device)

print(p.prob_factorized_text, p.prob_text)
print(_p.prob_factorized_text, _p.prob_text)
print(q.prob_factorized_text, q.prob_text)

In [ ]:
for param in p.parameters():
     print(type(param.data), param.size())

In [ ]:
model = VI(p, q, optim.Adam, {"lr":1e-3})

In [ ]:
def train(epoch):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        lower_bound, loss = model.train({"x": data.view(-1, 784)})
        train_loss += loss
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test(epoch):
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        lower_bound, loss = model.test({"x": data.view(-1, 784)})
        test_loss += loss
        z = q.sample({"x": data.view(-1, 784)})
        z = get_dict_values(z, _p.cond_var, return_dict=True)
        recon_batch = _p.sample_mean(z)
        if i == 0:
            n = min(data.size(0), 8)
            comparison = torch.cat([data[:n],
                                  recon_batch.view(batch_size, 1, 28, 28)[:n]])
            save_image(comparison.cpu(),
                     'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample_z = 0.3 * torch.randn(64, z_dim).to(device)
        sample = _p.sample_mean({"z":sample_z}).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')